In [ ]:
import numpy as np
from dcTMD.dcTMD import ForceEstimator
from dcTMD.storing import load

In [ ]:
## load force
# force = load('my_force_set')
# Instantiate a ForceEstimator instance and fit it with the ForceSet
# instance
force_estimator = ForceEstimator(temperature=290.15)
force_estimator